# EPI 270   

# Beginner DBDP Tutorial: Working with Continuous Glucose Monitoring (CGM) Data
# Credit: Brinnae Bent, PhD


In this case study, we will be using continuous glucose monitoring (CGM) data. We will be performing the first few steps of the Digital Biomarker Discvoery Pipeline: 

1. Pre-process the data
2. Exploratory data analysis
3. Feature Engineering (using the DBDP cgmquantify module)

### Level
Beginner. You have limited knowledge of Python/R. This tutorial is a step-by-step guide.

### System Requirements
* Python (3.0.0+) - If you don't yet have Python installed, please follow [this video tutorial](https://www.youtube.com/watch?v=YJC6ldI3hWk) on how to download Python through Anaconda

For beginners, I recommend using either JupyterLab or Visual Studio Code to interface with your notebooks.

### Objectives
* Learn how to work with CGM data
* Learn the first 3 steps of the digital biomarker discovery pipeline
* Compute glucose variability metrics on data

### Dataset
We will be using a small example dataset from a single person's week long CGM data. This data has been de-identified and time shifted.


_____

### Import libraries 
If you installed with Anaconda, some of these libaries should be built-in. 

You can check if you have them by running the next block of code.

If you get an error, install them in your terminal window using "pip install X" where X is the package missing

In [ ]:
import numpy as np
import pandas as pd 
import datetime as datetime
import matplotlib.pyplot as plt

### Pre-processing: Import and Clean CGM Data
In python, functions are used by library.function. Below, we are using the pandas library (which we nicknamed "pd" when we imported it above). We want to use the read_csv function within the pandas library.

If you added the data-set in your local folder (the one your code is in), you can proceed. You may need to change the filename to be your entire folder path and filename if you have the dataset elsewhere.

We are importing the data as a special type of data structure, the dataframe. Learn more about data structures and, in particular, dataframes [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html).

In [ ]:
data = pd.read_csv('test_file.csv') 

Let's take a peak at the data...

In [ ]:
data

In [ ]:
data[0:15]

Okay, the data is pretty messy. For today's purposes, we only care about the Timestamp and Glucose columns. We want to drop everything else from the dataset.

In [ ]:
# Instantiate a new dataframe. We will call this one "df"
df = pd.DataFrame() # This creates an empty data frame

# Pull timestamp and glucose columns into the new, empty dataframe. We will change their names to "DateTime" and "Glucose" 
df['DateTime'] = data['Timestamp (YYYY-MM-DDThh:mm:ss)']
# For glucose, we also want to change the data to a number. Right now, Python thinks the glucose column values are strings (you can think of a string as text). We will wrap the function in pd.to_numeric() to convert the column to numbers.
df['Glucose'] = pd.to_numeric(data['Glucose Value (mg/dL)'])

# The first 12 rows don't even have matching glucose + time values, so let's drop those
df.drop(df.index[:12], inplace=True)

Great! We now have a clean dataframe to work with. Check it out below!

In [ ]:
df

Now, let's deal with the DateTime. Remember how we had to change the glucose column to be numbers instead of strings? 

Well, we have to tell Python that the "DateTime" column is a datetime data type. 

In [ ]:
# We will use pd.to_datetime() to convert the Time column to a datetime object. We need to tell Python what the current format is. 
df['DateTime'] =  pd.to_datetime(df['DateTime'], format='%Y-%m-%dT%H:%M:%S')

# I would also like to make a new column, "Day", with just the Day of the DateTime column.
df['Day'] = df['DateTime'].dt.date

In [ ]:
df

Did you notice that your index (the bold numbers on the left side of the dataframe) starts at 12? That is because we removed the first 12 rows from the dataframe. Let's re-index the dataframe so the index will start at 0.

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

_____
### Exploratory Data Analysis
We will explore just a few components of Exploratory Data Analysis with CGM data. 

If we want to get a brief overview of our data, running the describe() function on our dataframe will give us a nice summary, including count, mean, standard deviation, median, minimum, and maximum.

In [ ]:
df.describe()

Let's start with a simple plot, so I can start to understand what is happening to my data over time. This will also enable me to see missing data. 

In [ ]:
# Designate a figure size and font size
plt.figure(figsize=(25,5))
plt.rcParams.update({'font.size': 20})

# Plot
plt.plot(df['DateTime'], df['Glucose'], '.', color = '#1f77b4')

It looks like we have a little bit of missing data, on 10-26 and on 10-31. That's an important thing to address, but not our focus today. 


Glucose excursions (very high and very low glucose values) are important for assessing glycemic health. Let's re-plot this, but including the mean and standard deviation of glucose on the plot.

In [ ]:
# Calculate the mean and standard deviation of glucose. We will draw 3 lines on the plot: The mean, 1 standard deviation above the mean, and 1 standard deviation below the mean.
glucose_mean = np.mean(df['Glucose'])
up = np.mean(df['Glucose']) + np.std(df['Glucose'])
dw = np.mean(df['Glucose']) - np.std(df['Glucose'])

# Same plot as above
plt.figure(figsize=(25,5))
plt.rcParams.update({'font.size': 20})
plt.plot(df['DateTime'], df['Glucose'], '.', color = '#1f77b4')

# Plot 3 horizontal lines
plt.axhline(y=glucose_mean, color='red', linestyle='-')
plt.axhline(y=up, color='pink', linestyle='-')
plt.axhline(y=dw, color='pink', linestyle='-')

# Add a label
plt.ylabel('Glucose')

Now, what if we wanted to see a smoothed graph that performed interpolation between points. We can use lowess smoothing to do this.

In [ ]:
# Import lowess from the statsmodels library
from statsmodels.nonparametric.smoothers_lowess import lowess

First, we will create a new data frame that is the smoothed version of our data:

In [ ]:
filteres = lowess(df['Glucose'], df['DateTime'], is_sorted=True, frac=0.015, it=0) #0.025
filtered = pd.to_datetime(filteres[:,0], format='%Y-%m-%dT%H:%M:%S') 

Next, we will plot the smoothed data:

In [ ]:
# Set sizes
plt.figure(figsize=(25,5))
plt.rcParams.update({'font.size': 20})

# Same plot as before
plt.plot(df['DateTime'], df['Glucose'], '.')

# Plot smoothed data
plt.plot(filtered, filteres[:,1], 'r')

#Labels
plt.ylabel('Glucose')

For more or less smoothing, you can change the "frac" argument in the lowess function above. It is currently set at 0.015. Why not try 0.03 and see what happens?

____________
In order to determine the distribution of Glucose in the dataset, let's plot a histogram using the matplotlib hist function.

For a list of all of the arguments to the matplotlib hist() function, check out the documentation [here](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist.html).

In [ ]:
# Set sizes
plt.figure(figsize=(10,10))
plt.rcParams.update({'font.size': 20})

# Plot histogram
plt.hist(df['Glucose'])

# Labels
plt.xlabel('Glucose')
plt.ylabel('Count')

_____
### Feature Engineering

Feature engineering is the process by which we engineer features that we can then use as inputs in models to predict an outcome. 

The process of Feature Engineering can be data-driven or domain-driven. We will do a couple of each here:

#### Data-driven Feature Engineering

Simple data-driven features include summary metrics. Let's compute those below:

In [ ]:
meanG = np.nanmean(df['Glucose'])
medianG = np.nanmedian(df['Glucose'])
minG = np.nanmin(df['Glucose'])
maxG = np.nanmax(df['Glucose'])
Q1G = np.nanpercentile(df['Glucose'], 25)
Q3G = np.nanpercentile(df['Glucose'], 75)

We can also compute metrics over each day:

In [ ]:
# Standard Deviation over all days
interdaysd = np.std(df['Glucose'])

# Standard Deviation for each day
intradaysd =[]

for i in pd.unique(df['Day']):
    intradaysd.append(np.std(df[df['Day']==i]["Glucose"]))
    
# We can find the average intraday standard deviation:
intradaysd_mean = np.mean(intradaysd)



If we examine both the interdaysd and the intradaysd_mean, we see that they are different!

In [ ]:
print(intradaysd_mean)
print(interdaysd)

#### Domain-driven Feature Engineering

Domain-driven feature engineering usually takes the most time but is often the most effective, especially when handling physiology-based metrics. Domain-driven feature engineering involves a literature review and a better understanding of the underlying physiology. 

Two metrics that are well known in diabetes literature to be indicative of glycemic health are the [glucose management index (GMI)](https://care.diabetesjournals.org/content/early/2018/09/17/dc18-1581) and the [J-index](https://diabetes.diabetesjournals.org/content/62/5/1398). Let's compute these features.

In [ ]:
J = 0.001*((np.mean(df['Glucose'])+np.std(df['Glucose']))**2)

In [ ]:
GMI = 3.31 + (0.02392*np.mean(df['Glucose']))

____
The DBDP CGMQuantify Module has 25+ pre-configured features that it computes. [cgmquanitfy module](https://github.com/DigitalBiomarkerDiscoveryPipeline/cgmquantify). The code is available [here](https://github.com/DigitalBiomarkerDiscoveryPipeline/cgmquantify/blob/master/cgmquantify/__init__.py).

You can also install the cgmquantify package and have access to its functions. (Remember to pip install the package. For full instructions on how to install it, see [here](https://github.com/DigitalBiomarkerDiscoveryPipeline/cgmquantify/blob/master/README.md))

In [ ]:
import cgmquantify as cgm

In [ ]:
data = cgm.importdexcom('test_file.csv')

print('interdaysd is: ' + str(cgm.interdaysd(data)))
print('interdaycv is: ' + str(cgm.interdaycv(data)))


# LAB QUESTION 1:

Do a literature search for eA1C and describe what it is, and why it may be a meaningful feature for these types of data. 

Calculate the eA1C for the sample. 



# LAB QUESTION 2:

Perform a literature search for the time in range (TIR) and time out of range (TOR) for diabetes and describe why this is a meaningful feature in diabetes management. 

Calculate the ratio of time in range (TIR) vs time out of range (TOR) 


# LAB QUESTION 3:

Perform a literature search related to continuous glucose monitoring and identify another feature that may be interesting to researchers or clinicians. Describe that features, and how it's calculated, and then implement the calulation below. 

# LAB EXERCISE 

This exercise can be done in any software you choose (cgmquantify is available on R), including in this Jupyter notebook. Keep in mind though, your instance will only last until it times out from non-use, or you close your browser. If you save your changes, you'll need to download the ipynb and then reupload it for it to work properly. 

Download the dataset located here: https://public.jaeb.org/dataset/559

The file you'll be interested in is entitled NonDiabDeviceCGM.csv

This is a dataset of CGM measurements from 200 non-diabetic individuals for up to 46 days. Your task is to use a new feature to group these participants into groups based on any hypothesis. You don't need to prove they are different, or run any statistical tests, just group them and then describe those groups using a few plots and visualizations. You can use any features from the CGMQuantify module, or any new features that you develop related to blood glucose levels. The zip file that you download will include many other files which describe the age, weight, height, gender, race, ethnicity, medications and pre-existing conditions of the participants Feel free to join these tables to your dataset and include these in your analysis, or even as the subject of your grouping. 

## Following the exercise, write a data reflection, addressing the following questions:

Proposed research question
Initial findings in data exploration (mean, median, SD, skew, etc.)
Overview of your Feature (why did you choose it? what does it tell you? how is it calculated?)
Description and rationale for your grouping
Observations from your analysis
Challenges in using the dataset
Conclusion on your research question
Related tables and figures
Copy of annotated code (feel free to print the Jupyter notebook)
Answers and code from the Lab Questions above. 

### Other Recommended Resources
* [Paper on glucose variability](https://journals.sagepub.com/doi/full/10.1177/1932296819826111)
 